# Train a CNN

In this notebook we will go through all the steps required to train a fully convolutional neural network. Because this takes a while and uses a lot of GPU RAM a separate command line script (`train_nn.py`) is also provided in the `src` directory.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Depending on your combination of package versions, this can raise a lot of TF warnings... 
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import *
import tensorflow.keras.backend as K
import seaborn as sns
import pickle
from src.score import *
from collections import OrderedDict

In [3]:
tf.__version__

'2.1.0'

In [4]:
def limit_mem():
    """By default TF uses all available GPU memory. This function prevents this."""
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    tf.compat.v1.Session(config=config)

In [5]:
limit_mem()

In [6]:
sns.set_style('darkgrid')
sns.set_context('notebook')

In [7]:
DATADIR = '/rds/general/user/mc4117/home/WeatherBench/data/'

## Create data generator

First up, we want to write our own Keras data generator. The key advantage to just feeding in numpy arrays is that we don't have to load the data twice because our intputs and outputs are the same data just offset by the lead time. Since the dataset is quite large and we might run out of CPU RAM this is important.

In [39]:
# Load the validation subset of the data: 2017 and 2018
z500_valid = load_test_data(f'{DATADIR}geopotential_500', 'z')
t850_valid = load_test_data(f'{DATADIR}temperature_850', 't')
valid = xr.merge([z500_valid, t850_valid])

In [40]:
z = xr.open_mfdataset(f'{DATADIR}geopotential_500/*.nc', combine='by_coords')
t = xr.open_mfdataset(f'{DATADIR}temperature_850/*.nc', combine='by_coords').drop('level')

In [41]:
# For the data generator all variables have to be merged into a single dataset.
datasets = [z, t]
ds = xr.merge(datasets)

In [42]:
ds_train = ds.sel(time=slice('1979', '2016'))  
ds_test = ds.sel(time=slice('2017', '2018'))

In [43]:
"""
data_z = ds_train.z
bins_z = np.linspace(ds_train.z.min(), ds_train.z.max(), 200)
digitized_z = np.digitize(data_z, bins_z)-1

binned_data_z = np.zeros(digitized_z.shape)
nx, ny, nz = digitized_z.shape

for i in range(nx):
    for j in range(ny):
        for k in range(nz):
            binned_data_z[i, j, k] = bins_z[digitized_z[i, j, k]]
            
data_t = ds_train.t
bins_t = np.linspace(ds_train.t.min(), ds_train.t.max(), 200)
digitized_t = np.digitize(data_t, bins_t)-1

binned_data_t = np.zeros(digitized_t.shape)
nx, ny, nz = digitized_t.shape

for i in range(nx):
    for j in range(ny):
        for k in range(nz):
            binned_data_t[i, j, k] = bins_t[digitized_t[i, j, k]]

ds_train_binned = ds_train.copy()
ds_train_binned.z.values = binned_data_z
ds_train_binned.t.values = binned_data_t
"""

'\ndata_z = ds_train.z\nbins_z = np.linspace(ds_train.z.min(), ds_train.z.max(), 200)\ndigitized_z = np.digitize(data_z, bins_z)-1\n\nbinned_data_z = np.zeros(digitized_z.shape)\nnx, ny, nz = digitized_z.shape\n\nfor i in range(nx):\n    for j in range(ny):\n        for k in range(nz):\n            binned_data_z[i, j, k] = bins_z[digitized_z[i, j, k]]\n            \ndata_t = ds_train.t\nbins_t = np.linspace(ds_train.t.min(), ds_train.t.max(), 200)\ndigitized_t = np.digitize(data_t, bins_t)-1\n\nbinned_data_t = np.zeros(digitized_t.shape)\nnx, ny, nz = digitized_t.shape\n\nfor i in range(nx):\n    for j in range(ny):\n        for k in range(nz):\n            binned_data_t[i, j, k] = bins_t[digitized_t[i, j, k]]\n\nds_train_binned = ds_train.copy()\nds_train_binned.z.values = binned_data_z\nds_train_binned.t.values = binned_data_t\n'

In [44]:
"""
error_z = binned_data_z - ds.z
weights_lat = np.cos(np.deg2rad(error_z.lat))
weights_lat /= weights_lat.mean()
rmse_z = np.sqrt(((error_z)**2 * weights_lat).mean(mean_dims))
print(rmse_z.values)

error_t = binned_data_t - ds.t
weights_lat = np.cos(np.deg2rad(error_t.lat))
weights_lat /= weights_lat.mean()
rmse_t = np.sqrt(((error_t)**2 * weights_lat).mean(mean_dims))
print(rmse_t.values)
"""

'\nerror_z = binned_data_z - ds.z\nweights_lat = np.cos(np.deg2rad(error_z.lat))\nweights_lat /= weights_lat.mean()\nrmse_z = np.sqrt(((error_z)**2 * weights_lat).mean(mean_dims))\nprint(rmse_z.values)\n\nerror_t = binned_data_t - ds.t\nweights_lat = np.cos(np.deg2rad(error_t.lat))\nweights_lat /= weights_lat.mean()\nrmse_t = np.sqrt(((error_t)**2 * weights_lat).mean(mean_dims))\nprint(rmse_t.values)\n'

In [45]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, ds, var_dict, lead_time, batch_size=32, shuffle=True, load=True, mean=None, std=None):
        """
        Data generator for WeatherBench data.
        Template from https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
        Args:
            ds: Dataset containing all variables
            var_dict: Dictionary of the form {'var': level}. Use None for level if data is of single level
            lead_time: Lead time in hours
            batch_size: Batch size
            shuffle: bool. If True, data is shuffled.
            load: bool. If True, datadet is loaded into RAM.
            mean: If None, compute mean from data.
            std: If None, compute standard deviation from data.
        """
        self.ds = ds
        self.var_dict = var_dict
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.lead_time = lead_time

        data = []
        generic_level = xr.DataArray([1], coords={'level': [1]}, dims=['level'])
        for var, levels in var_dict.items():
            try:
                data.append(ds[var].sel(level=levels))
            except ValueError:
                data.append(ds[var].expand_dims({'level': generic_level}, 1))

        self.data = xr.concat(data, 'level').transpose('time', 'lat', 'lon', 'level')
        self.mean = self.data.mean(('time', 'lat', 'lon')).compute() if mean is None else mean
        self.std = self.data.std('time').mean(('lat', 'lon')).compute() if std is None else std
        # Normalize
        self.data = (self.data - self.mean) / self.std
        self.n_samples = self.data.isel(time=slice(0, -lead_time)).shape[0]
        self.init_time = self.data.isel(time=slice(None, -lead_time)).time
        self.valid_time = self.data.isel(time=slice(lead_time, None)).time
        
        
        bins_z = np.linspace(ds.z.min(), ds.z.max(), 200)
        digitized_z = np.digitize(ds.z, bins_z)-1

        binned_data_z = np.zeros(digitized_z.shape)
        nx, ny, nz = digitized_z.shape

        for i in range(nx):
            for j in range(ny):
                for k in range(nz):
                    binned_data_z[i, j, k] = bins_z[digitized_z[i, j, k]]

        bins_t = np.linspace(ds.t.min(), ds.t.max(), 200)
        digitized_t = np.digitize(ds.t, bins_t)-1

        binned_data_t = np.zeros(digitized_t.shape)
        nx, ny, nz = digitized_t.shape

        for i in range(nx):
            for j in range(ny):
                for k in range(nz):
                    binned_data_t[i, j, k] = bins_t[digitized_t[i, j, k]]
        ds_binned = ds.copy()
        ds_binned.z.values = binned_data_z
        ds_binned.t.values = binned_data_t
        
        binned_data = []
        generic_level = xr.DataArray([1], coords={'level': [1]}, dims=['level'])
        for var, levels in var_dict.items():
            try:
                binned_data.append(ds_binned[var].sel(level=levels))
            except ValueError:
                binned_data.append(ds_binned[var].expand_dims({'level': generic_level}, 1))

        self.binned_data = xr.concat(binned_data, 'level').transpose('time', 'lat', 'lon', 'level')
        # Normalize
        self.binned_data = (self.binned_data - self.mean) / self.std        

        self.on_epoch_end()

        # For some weird reason calling .load() earlier messes up the mean and std computations
        if load: print('Loading data into RAM'); self.data.load()
        if load: print('Loading data into RAM'); self.binned_data.load()            

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.ceil(self.n_samples / self.batch_size))

    def __getitem__(self, i):
        'Generate one batch of data'
        idxs = self.idxs[i * self.batch_size:(i + 1) * self.batch_size]
        X = self.data.isel(time=idxs).values
        y = self.binned_data.isel(time=idxs + self.lead_time).values
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.idxs = np.arange(self.n_samples)
        if self.shuffle == True:
            np.random.shuffle(self.idxs)

In [46]:
# then we need a dictionary for all the variables and levels we want to extract from the dataset
dic = OrderedDict({'z': None, 't': None})

In [47]:
bs=32
lead_time=72

In [48]:
# Create a training and validation data generator. Use the train mean and std for validation as well.
dg_train = DataGenerator(
    ds_train.sel(time=slice('1979', '2015')), dic, lead_time, batch_size=bs, load=True)

dg_valid = DataGenerator(
    ds_train.sel(time=slice('2016', '2016')), dic, lead_time, batch_size=bs, mean=dg_train.mean, std=dg_train.std, shuffle=False)

dg_test = DataGenerator(
    ds_test, dic, lead_time, batch_size=bs, mean=dg_train.mean, std=dg_train.std, shuffle=False)

Loading data into RAM
Loading data into RAM
Loading data into RAM
Loading data into RAM
Loading data into RAM
Loading data into RAM


## Create and train model

Next up, we need to create the model architecture. Here we will use a fully connected convolutional network. Because the Earth is periodic in longitude, we want to use a periodic convolution in the lon-direction. This is not implemented in Keras, so we have to do it manually.

In [49]:
class PeriodicPadding2D(tf.keras.layers.Layer):
    def __init__(self, pad_width, **kwargs):
        super().__init__(**kwargs)
        self.pad_width = pad_width

    def call(self, inputs, **kwargs):
        if self.pad_width == 0:
            return inputs
        inputs_padded = tf.concat(
            [inputs[:, :, -self.pad_width:, :], inputs, inputs[:, :, :self.pad_width, :]], axis=2)
        # Zero padding in the lat direction
        inputs_padded = tf.pad(inputs_padded, [[0, 0], [self.pad_width, self.pad_width], [0, 0], [0, 0]])
        return inputs_padded

    def get_config(self):
        config = super().get_config()
        config.update({'pad_width': self.pad_width})
        return config


class PeriodicConv2D(tf.keras.layers.Layer):
    def __init__(self, filters,
                 kernel_size,
                 conv_kwargs={},
                 **kwargs, ):
        super().__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.conv_kwargs = conv_kwargs
        if type(kernel_size) is not int:
            assert kernel_size[0] == kernel_size[1], 'PeriodicConv2D only works for square kernels'
            kernel_size = kernel_size[0]
        pad_width = (kernel_size - 1) // 2
        self.padding = PeriodicPadding2D(pad_width)
        self.conv = Conv2D(
            filters, kernel_size, padding='valid', **conv_kwargs
        )

    def call(self, inputs):
        return self.conv(self.padding(inputs))

    def get_config(self):
        config = super().get_config()
        config.update({'filters': self.filters, 'kernel_size': self.kernel_size, 'conv_kwargs': self.conv_kwargs})
        return config

In [50]:
def build_cnn(filters, kernels, input_shape, dr=0):
    """Fully convolutional network"""
    x = input = Input(shape=input_shape)
    for f, k in zip(filters[:-1], kernels[:-1]):
        x = PeriodicConv2D(f, k)(x)
        x = LeakyReLU()(x)
        if dr > 0: x = Dropout(dr)(x)
    output = PeriodicConv2D(filters[-1], kernels[-1])(x)
    return keras.models.Model(input, output)

In [51]:
cnn = build_cnn([64, 64, 64, 64, 2], [5, 5, 5, 5, 5], (32, 64, 2))

In [24]:
cnn.compile(keras.optimizers.Adam(1e-4), 'mse')

In [25]:
cnn.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 64, 2)]       0         
_________________________________________________________________
periodic_conv2d (PeriodicCon (None, 32, 64, 64)        3264      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32, 64, 64)        0         
_________________________________________________________________
periodic_conv2d_1 (PeriodicC (None, 32, 64, 64)        102464    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 64, 64)        0         
_________________________________________________________________
periodic_conv2d_2 (PeriodicC (None, 32, 64, 64)        102464    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 32, 64, 64)        0     

In [26]:
# python_files/whole_orig_nn_72.py

cnn.fit(dg_train, epochs=100, validation_data=dg_valid, 
          callbacks=[tf.keras.callbacks.EarlyStopping(
                        monitor='val_loss',
                        min_delta=0,
                        patience=2,
                        verbose=1, 
                        mode='auto'
                    )]
         )

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 10134 steps, validate for 273 steps
Epoch 1/100
  225/10134 [..............................] - ETA: 4:14:33 - loss: 0.8774WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



KeyboardInterrupt: 

In [ ]:
# Load weights from externally trained model
cnn.load_weights('/rds/general/user/mc4117/home/WeatherBench/saved_models/whole_train_72.h5')

## Create predictions

Now that we have our model we need to create a prediction NetCDF file. This function does this. 

We can either directly predict the target lead time (e.g. 5 days) or create an iterative forecast by chaining together many e.g. 6h forecasts.

In [ ]:
def create_predictions(model, dg):
    """Create predictions for non-iterative model"""
    preds = model.predict_generator(dg)
    # Unnormalize
    preds = preds * dg.std.values + dg.mean.values
    fcs = []
    lev_idx = 0
    for var, levels in OrderedDict({'z': None, 't': None}).items():
        if levels is None:
            fcs.append(xr.DataArray(
                preds[:, :, :, lev_idx],
                dims=['time', 'lat', 'lon'],
                coords={'time': dg.valid_time, 'lat': dg.ds.lat, 'lon': dg.ds.lon},
                name=var
            ))
            lev_idx += 1
        else:
            nlevs = len(levels)
            fcs.append(xr.DataArray(
                preds[:, :, :, lev_idx:lev_idx+nlevs],
                dims=['time', 'lat', 'lon', 'level'],
                coords={'time': dg.valid_time, 'lat': dg.ds.lat, 'lon': dg.ds.lon, 'level': levels},
                name=var
            ))
            lev_idx += nlevs
    return xr.merge(fcs)

In [ ]:
fc = create_predictions(cnn, dg_test)

In [ ]:
cnn_rmse = compute_weighted_rmse(fc, valid)

In [ ]:
compute_weighted_rmse(fc, valid).compute()

In [ ]:
PREDDIR = '/rds/general/user/mc4117/home/WeatherBench/baselines/'
fc.to_netcdf(f'{PREDDIR}/cnn_whole_72.nc')